In [1]:
import nemo.collections.asr as nemo_asr
import os



In [2]:
from flwr.common import Scalar
import flwr as fl
import torch
import numpy as np
from typing import Dict, Tuple, List
from collections import OrderedDict
import pytorch_lightning as ptl
from omegaconf import open_dict

def count_lines(file_path):
    with open(file_path, 'r') as file:
        line_count = sum(1 for line in file)
    return line_count

def set_parameters(net: torch.nn.Module, parameters: List[np.ndarray]) -> None:
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)
        
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_config(net, train_manifest_path, test_manifest_path):
    cfg = net.cfg
    with open_dict(cfg):
        cfg.train_ds.manifest_filepath = train_manifest_path
        cfg.train_ds.batch_size = 1
        cfg.train_ds.num_workers = 8
        cfg.train_ds.is_tarred = False
        #   cfg.train_ds.pin_memory=True \
        cfg.test_ds.manifest_filepath = test_manifest_path
        cfg.test_ds.batch_size = 1
        cfg.test_ds.num_workers = 8

        # Validation dataset  (Use test dataset as validation, since we train using train + dev)
        cfg.validation_ds.manifest_filepath = test_manifest_path
        cfg.validation_ds.batch_size = 1
        cfg.validation_ds.num_workers = 8
        cfg.train_ds.is_tarred = False

        cfg.spec_augment.freq_mask: 0
        cfg.spec_augment.time_mask: 0
        
        # Optimizer and Scheduler
        cfg.optim.name = 'adamw'
        cfg.optim.lr = 1.0e-4
        cfg.optim.betas = [0.9, 0.98]
        cfg.optim.weight_decay = 0.0005
        cfg.optim.sched.max_steps = (10 // 1) * 1
        cfg.optim.sched.name = "CosineAnnealing"
        cfg.optim.sched.min_lr = 1.0e-7
        cfg.optim.sched.warmup_steps = 0
            # Remove unnecessary parameters
        if 'warmup_ratio' in cfg.optim.sched:
            del cfg.optim.sched['warmup_ratio']
        if 'hold_ratio' in cfg.optim.sched:
            del cfg.optim.sched['hold_ratio']
        if 'hold_steps' in cfg.optim.sched:
            del cfg.optim.sched['hold_steps']
        if 'decay_rate' in cfg.optim.sched:
            del cfg.optim.sched['decay_rate']
    
    net.cfg = net._cfg
    net.setup_training_data(cfg.train_ds)
    net.setup_validation_data(cfg.validation_ds)
    net.setup_optimization(cfg.optim)
    net.spec_augmentation = net.from_config_dict(cfg.spec_augment)
    

class NemoASRClient(fl.client.NumPyClient):
    def __init__(self, model, train_manifest_path, test_manifest_path):
        set_config(model, train_manifest_path, test_manifest_path)
        self.model = model
        if torch.cuda.is_available():
          accelerator = 'gpu'
        else:
          accelerator = 'cpu'
        print(f"Using {self.model.cfg.validation_ds.manifest_filepath} testdatamanifest")
        trainer = ptl.Trainer(devices=1,
                              accelerator=accelerator,
                              max_epochs=1,
                              accumulate_grad_batches=1,
                              enable_checkpointing=False,
                              logger=False,
                              log_every_n_steps=1,
                              check_val_every_n_epoch=2)
        self.trainer = trainer

        self.model.set_trainer(trainer)


    def get_parameters(self, config: Dict[str, Scalar]) -> List[np.ndarray]:
        return get_parameters(self.model)
    
    def fit(
        self, parameters: List[np.ndarray], config: Dict[str, Scalar]
    ) -> Tuple[List[np.ndarray], int, Dict[str, Scalar]]:
        set_parameters(self.model, parameters)
        self.trainer.fit(self.model)
        # params, num train samples, other metrics
        return get_parameters(self.model), count_lines(self.model.cfg.train_ds.manifest_filepath), {}

    def evaluate(
            self, parameters: List[np.ndarray], config: Dict[str, Scalar]
        ) -> Tuple[float, int, Dict[str, Scalar]]:
        
        if config.get("evaluate"):
            set_parameters(self.model, parameters)
            test_results = self.trainer.validate(self.model)
            test_result = test_results[0]
            loss = test_result.get('test_loss', 0.0)
            wer = test_result.get('test_wer', 0.0) 
            return loss, count_lines(self.model.cfg.validation_ds.manifest_filepath), {"wer": wer}
        return 0, 0, {}


In [3]:
from collections import defaultdict
import json
import os
def create_speaker_manifests(train_manifest_path, output_dir):
    with open(train_manifest_path, 'r') as f:
        lines = f.readlines()
    
    speaker_data = defaultdict(list)
    
    for line in lines:
        data = json.loads(line)
        speaker_id = data['audio_filepath'].split('/')[-2]
        speaker_data[speaker_id].append(data)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    llspeaker_manifest_paths = set()
    for speaker_id, data in speaker_data.items():
        speaker_manifest_path = os.path.join(output_dir, f"{speaker_id}_manifest.json")
        with open(speaker_manifest_path, 'w') as f:
            for entry in data:
                f.write(json.dumps(entry) + '\n')
        llspeaker_manifest_paths.add(speaker_manifest_path)
    
    return llspeaker_manifest_paths

fed_manifests_folder_path = "timit-dataset/federated-manifests"
# Example usage
speaker_manifest_paths = create_speaker_manifests("timit-dataset/train-manifest.clean.json", fed_manifests_folder_path)

In [4]:
def client_fn(cid: str):
    asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_squeezeformer_ctc_xsmall_ls")
    
    # Pop a manifest path from the set
    train_manifest_path = speaker_manifest_paths.pop()
    test_manifest_path = "timit-dataset/test-manifest.clean.json"
    
    #return NemoASRClient(asr_model, train_manifest_path, test_manifest_path).to_client()
    return NemoASRClient(asr_model, train_manifest_path, train_manifest_path).to_client()



In [5]:
import flwr as fl


def on_fit_config_fn(rnd: int):
    return {"epoch": rnd}

def on_evaluate_config_fn(rnd: int):
    return {"epoch": rnd, "evaluate": True}

def evaluate_fn(server_round, parameters, config):
    # Load global model
    global_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_squeezeformer_ctc_xsmall_ls")
    set_config(global_model, "timit-dataset/train-manifest.clean.json", "timit-dataset/test-manifest.clean.json")
    set_parameters(global_model, parameters)
    
    # Perform evaluation
    trainer = ptl.Trainer(devices=1, accelerator='gpu' if torch.cuda.is_available() else 'cpu')
    results = trainer.validate(global_model)
    loss = results[0].get('test_loss', 0.0)
    wer = results[0].get('test_wer', 0.0)

    return loss, {"wer": wer}
    #return 0, {"wer": 0}

strategy = fl.server.strategy.FedAvg(
    on_fit_config_fn=on_fit_config_fn,
    on_evaluate_config_fn=on_evaluate_config_fn,
    evaluate_fn=evaluate_fn
)

#num_clients = len(speaker_manifest_paths)
num_clients = 2
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=num_clients,
    client_resources={"num_cpus": 1, "num_gpus": 1},  # Adjust as needed
    config=fl.server.ServerConfig(num_rounds=1),
    strategy=strategy,
)


INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-07-12 22:16:23,540	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 4154695680.0, 'CPU': 8.0, 'node:172.26.121.70': 1.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'accelerator_type:G': 1.0, 'object_store_memory': 2077347840.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=27786) /home/robert/.virtualenvs/ForschungsprojektA/lib/python3.10/site-packages/lightning_utilities/core/imports.py:14: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resourc

(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:33 cloud:58] Found existing object /home/robert/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_squeezeformer_ctc_xsmall_ls/4d9c4c04221bedd8427635f462c97561/stt_en_squeezeformer_ctc_xsmall_ls.nemo.
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:33 cloud:64] Re-using file from: /home/robert/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_squeezeformer_ctc_xsmall_ls/4d9c4c04221bedd8427635f462c97561/stt_en_squeezeformer_ctc_xsmall_ls.nemo
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:33 common:815] Instantiating model from pre-trained checkpoint


(ClientAppActor pid=27786) [NeMo W 2024-07-12 22:16:34 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
(ClientAppActor pid=27786)     Train config : 
(ClientAppActor pid=27786)     manifest_filepath: /data/tarred_train/tarred_audio_manifest.json
(ClientAppActor pid=27786)     sample_rate: 16000
(ClientAppActor pid=27786)     batch_size: 32
(ClientAppActor pid=27786)     shuffle: true
(ClientAppActor pid=27786)     num_workers: 8
(ClientAppActor pid=27786)     pin_memory: true
(ClientAppActor pid=27786)     use_start_end_token: false
(ClientAppActor pid=27786)     trim_silence: false
(ClientAppActor pid=27786)     max_duration: 16.7
(ClientAppActor pid=27786)     min_duration: 0.1
(ClientAppActor pid=27786)     is_tarred: true
(ClientAppActor pid=27786)     tarred_audio_filepaths: /data/tarred_train/audio__OP_0..511_CL_.tar
(ClientAppActor pid=27786)     

(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:34 mixins:172] Tokenizer SentencePieceTokenizer initialized with 128 tokens
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:34 features:305] PADDING: 0
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:34 save_restore_connector:263] Model EncDecCTCModelBPE was successfully restored from /home/robert/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_squeezeformer_ctc_xsmall_ls/4d9c4c04221bedd8427635f462c97561/stt_en_squeezeformer_ctc_xsmall_ls.nemo.
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:34 collections:196] Dataset loaded with 10 files totalling 0.01 hours
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:34 collections:197] 0 files were filtered totalling 0.00 hours
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:34 collections:196] Dataset loaded with 10 files totalling 0.01 hours
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:16:34 collections:197] 0 files were filtered totalling 0.00 hours
(ClientAppActor pid=27786)

(ClientAppActor pid=27786) [NeMo W 2024-07-12 22:16:34 modelPT:652] Trainer wasn't specified in model constructor. Make sure that you really wanted it.
(ClientAppActor pid=27786) GPU available: True (cuda), used: True
(ClientAppActor pid=27786) TPU available: False, using: 0 TPU cores
(ClientAppActor pid=27786) HPU available: False, using: 0 HPUs
INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters


...

(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:14 save_restore_connector:263] Model EncDecCTCModelBPE was successfully restored from /home/robert/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_squeezeformer_ctc_xsmall_ls/4d9c4c04221bedd8427635f462c97561/stt_en_squeezeformer_ctc_xsmall_ls.nemo.
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:14 collections:196] Dataset loaded with 10 files totalling 0.01 hours
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:14 collections:197] 0 files were filtered totalling 0.00 hours
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:14 collections:196] Dataset loaded with 10 files totalling 0.01 hours
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:14 collections:197] 0 files were filtered totalling 0.00 hours
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:14 modelPT:770] Optimizer config = AdamW (
(ClientAppActor pid=27786)     Parameter Group 0
(ClientAppActor pid=27786)         amsgrad: False
(ClientAppActor pid=27786)         betas

(ClientAppActor pid=27786) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/10 [00:00<?, ?it/s]
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:15 wer:334] 
(ClientAppActor pid=27786)     
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:15 wer:335] reference:she had your dark suit in greasy wash water all year
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:15 wer:336] predicted:she had your dark suit in greasy wash water all year
Validation DataLoader 0:  10%|█         | 1/10 [00:00<00:01,  8.62it/s]
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:15 wer:334] 
(ClientAppActor pid=27786)     
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:15 wer:335] reference:don't ask me to carry an oily rag like that
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:15 wer:336] predicted:don't ask me to carry an oily rag like that
Validation DataLoader 0:  20%|██        | 2/10 [00:00<00:00, 10.55it/s]
(ClientAppActor pid=27786) [NeMo I 2024-07-12 22:21:15 wer:334

INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 1 round(s) in 147.74s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.0
INFO :      		round 1: 0.0
INFO :      	History (metrics, centralized):
INFO :      	{'wer': [(0, 0.0), (1, 0.0)]}
INFO :      


History (loss, distributed):
	round 1: 0.0
History (loss, centralized):
	round 0: 0.0
	round 1: 0.0
History (metrics, centralized):
{'wer': [(0, 0.0), (1, 0.0)]}